# Análisis de Sentimiento con una Red LSTM usando Keras

## Objetivo

En esta actividad vas a construir un modelo de red neuronal recurrente (RNN), específicamente una LSTM, usando la API Keras de TensorFlow. El modelo va a leer frases en español y clasificar su sentimiento como positivo o negativo.

### ¿Qué vamos a lograr?

- Entender cómo las redes recurrentes procesan secuencias de palabras
- Implementar una LSTM que recuerda el contexto de la frase
- Usar embeddings de palabras para representar el significado
- Observar cómo las LSTM superan las limitaciones de bag-of-words

### ¿Qué es una LSTM?

LSTM (Long Short-Term Memory) es un tipo especial de red neuronal recurrente diseñada para:
- **Procesar secuencias**: Lee las palabras en orden, una después de la otra
- **Mantener memoria**: Recuerda información importante de palabras anteriores
- **Olvidar información irrelevante**: Decide qué información del pasado mantener y qué descartar

A diferencia de las MLP que vimos antes, las LSTM **sí consideran el orden** de las palabras, lo cual es fundamental para entender el lenguaje.

## 1. Preparación del entorno

Importamos las librerías necesarias, incluyendo herramientas de Keras para procesamiento de secuencias.

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(f"TensorFlow/Keras versión: {keras.__version__}")

## 2. Datos de entrenamiento

Vamos a usar las mismas frases que en la actividad anterior, pero ahora las vamos a procesar como **secuencias de palabras**, no como bolsa de palabras.

Esta diferencia es fundamental: la LSTM va a poder aprovechar el orden en que aparecen las palabras.

In [ ]:
frases = [
    "La verdad, este lugar está bárbaro. Muy recomendable",
    "Una porquería de servicio, nunca más vuelvo",
    "Me encantó la comida, aunque la música estaba muy fuerte",
    "El envío fue lento y el producto llegó dañado. Qué desastre",
    "Todo excelente. Atención de diez",
    "Qué estafa, me arrepiento de haber comprado",
    "Muy conforme con el resultado final",
    "No me gustó para nada la experiencia",
    "Superó mis expectativas, gracias",
    "No lo recomiendo, mala calidad"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

print(f"Total de frases: {len(frases)}")
print(f"Balance: {sum(etiquetas)} positivas, {len(etiquetas) - sum(etiquetas)} negativas\n")
print("Ejemplos:")
for i in range(3):
    sentimiento = "Positivo" if etiquetas[i] == 1 else "Negativo"
    print(f"  {i+1}. '{frases[i][:50]}...' → {sentimiento}")

## 3. Tokenización y construcción del vocabulario

Con Keras, vamos a convertir las frases en secuencias de números, donde cada número representa una palabra del vocabulario.

### Diferencia clave con bag-of-words:

**Bag-of-words:**
- "Me gusta" → [1, 0, 1, 0, 0] (solo presencia/ausencia)

**Secuencia:**
- "Me gusta" → [5, 12] (orden preservado, cada palabra tiene un ID)

In [ ]:
# Tokenización: convierte palabras a números
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(frases)

# Mostramos el vocabulario construido
vocab_size = len(tokenizer.word_index) + 1  # +1 por el índice 0
print(f"Tamaño del vocabulario: {vocab_size} palabras únicas\n")
print("Primeras 15 palabras del vocabulario:")
for palabra, idx in list(tokenizer.word_index.items())[:15]:
    print(f"  '{palabra}' → {idx}")

# Convertimos frases a secuencias numéricas
secuencias = tokenizer.texts_to_sequences(frases)

print(f"\nEjemplo de conversión:")
print(f"Frase original: '{frases[0]}'")
print(f"Secuencia numérica: {secuencias[0]}")

## 4. Padding: estandarizando la longitud de las secuencias

Las redes neuronales necesitan entradas de tamaño fijo, pero nuestras frases tienen longitudes diferentes.

**Solución: Padding**
- Rellenamos las secuencias cortas con ceros al final
- Todas las secuencias terminan con la misma longitud

Ejemplo:
- `[5, 12]` → `[5, 12, 0, 0, 0]` (padding='post')

In [ ]:
# Calculamos la longitud máxima
maxlen = max(len(seq) for seq in secuencias)
print(f"Longitud de la frase más larga: {maxlen} palabras\n")

# Aplicamos padding
X = pad_sequences(secuencias, maxlen=maxlen, padding='post')
y = np.array(etiquetas)

print(f"Forma de X después del padding: {X.shape}")
print(f"  {X.shape[0]} frases × {X.shape[1]} posiciones\n")

print("Ejemplo de secuencia con padding:")
print(f"Frase: '{frases[0]}'")
print(f"Secuencia: {X[0]}")
print(f"Nota: Los ceros al final son padding (relleno)")

## 5. Definición del modelo LSTM

Vamos a construir una red con tres componentes clave:

### 1. Capa de Embedding
Convierte cada palabra (número) en un vector denso de dimensión fija. Estos vectores se aprenden durante el entrenamiento y capturan similitudes semánticas.

**Ejemplo conceptual:**
- "excelente" → [0.8, 0.9, -0.1, 0.7, ...]
- "bueno" → [0.7, 0.8, -0.2, 0.6, ...] (vector similar)
- "malo" → [-0.7, -0.8, 0.2, -0.6, ...] (vector opuesto)

### 2. Capa LSTM
Procesa la secuencia de embeddings manteniendo memoria del contexto.

### 3. Capa Dense (salida)
Clasifica el sentimiento basándose en la representación aprendida por la LSTM.

In [ ]:
###############################################################################
# CONFIGURACIÓN DE LA ARQUITECTURA LSTM
###############################################################################

###############################################################################
# PARÁMETRO 1: embedding_dim (Dimensión de los embeddings)
###############################################################################
# ¿Qué es embedding_dim?
# Es el tamaño del vector que representa cada palabra.
#
# Ejemplo conceptual con embedding_dim=16:
# "excelente" → [0.8, 0.3, -0.5, 0.1, ..., 0.7]  (16 números)
# "bueno"     → [0.7, 0.2, -0.4, 0.0, ..., 0.6]  (16 números, similar)
# "malo"      → [-0.6, -0.3, 0.4, -0.2, ..., -0.5] (16 números, opuesto)
#
# ¿Por qué 16?
# - Vocabulario pequeño (~50 palabras): 16 dimensiones son suficientes
# - Valores típicos: 50, 100, 300 para vocabularios grandes
# - Modelos grandes como Word2Vec: 300 dimensiones
# - Modelos masivos como GPT-3: 12,288 dimensiones
#
# ¿Qué pasa si cambio embedding_dim?
# - Muy pequeño (ej: 4): No captura suficientes matices del significado
# - Muy grande (ej: 512): Overfitting con pocos datos de entrenamiento
# - Regla práctica: Entre 16 y 100 para datasets pequeños
#
# Diferencia con bag-of-words:
# - Bag-of-words: Vector sparse de tamaño=vocab_size, casi todo ceros
#   Ejemplo: [0, 0, 1, 0, 0, ..., 0] (50 posiciones, 49 ceros)
# - Embedding: Vector denso de tamaño=embedding_dim, todos valores útiles
#   Ejemplo: [0.8, 0.3, -0.5, ..., 0.7] (16 valores significativos)
embedding_dim = 16

###############################################################################
# PARÁMETRO 2: lstm_units (Unidades en la capa LSTM)
###############################################################################
# ¿Qué es lstm_units?
# Es el tamaño del "estado oculto" de la LSTM.
# Determina cuánta información puede "recordar" la red.
#
# Internamente, la LSTM tiene 4 componentes por unidad:
# 1. Input gate: Qué información nueva agregar
# 2. Forget gate: Qué información olvidar
# 3. Output gate: Qué información sacar
# 4. Cell state: Memoria a largo plazo
#
# ¿Por qué 32 unidades?
# - Es el doble del embedding_dim (16 × 2 = 32)
# - Valor típico para secuencias cortas
# - Suficiente para capturar patrones en frases de ~10 palabras
#
# ¿Qué pasa si cambio lstm_units?
# - Muy pequeño (ej: 8): Poca capacidad de memoria
# - Muy grande (ej: 512): Overfitting, y entrenamiento lento
# - Valores típicos: 32, 64, 128, 256
#
# Comparación de parámetros:
# Con 32 unidades, la capa LSTM tiene aproximadamente:
# 4 × lstm_units × (lstm_units + embedding_dim + 1) parámetros
# = 4 × 32 × (32 + 16 + 1) = 4 × 32 × 49 = 6,272 parámetros
#
# Mucho más que la MLP del laboratorio anterior (145 parámetros)
lstm_units = 32

###############################################################################
# CONSTRUCCIÓN DEL MODELO SECUENCIAL
###############################################################################
# Sequential permite apilar capas una tras otra
# Flujo de datos: Input → Embedding → LSTM → Dense → Output
modelo = Sequential([
    
    ###########################################################################
    # CAPA 1: Embedding (Representación de palabras)
    ###########################################################################
    # Embedding(input_dim, output_dim, input_length)
    #
    # ¿Qué hace esta capa?
    # Convierte cada palabra (número entero) en un vector denso de números reales.
    #
    # Parámetros:
    # - input_dim=vocab_size: Tamaño del vocabulario (~50 palabras únicas)
    # - output_dim=embedding_dim: Dimensión del vector (16)
    # - input_length=maxlen: Longitud de las secuencias (~10 palabras)
    #
    # ¿Cómo se inicializan los embeddings?
    # Keras usa inicialización uniforme aleatoria por defecto:
    # Valores entre -0.05 y +0.05
    #
    # ¿Cómo se aprenden?
    # Durante backpropagation, los vectores se ajustan para que:
    # - Palabras con sentimiento similar tengan vectores cercanos
    # - Palabras positivas y negativas estén separadas en el espacio vectorial
    #
    # Ejemplo de transformación:
    # Input:  [5, 12, 0, 0, ...]  (secuencia de IDs de palabras)
    # Output: [[0.8, 0.3, ...],    (vector de "palabra 5")
    #          [0.2, -0.5, ...],   (vector de "palabra 12")
    #          [0.0, 0.0, ...],    (vector de padding)
    #          ...]
    #
    # Forma de salida: (batch_size, maxlen, embedding_dim)
    #                  (batch, 10, 16)
    #
    # Parámetros totales: vocab_size × embedding_dim ≈ 50 × 16 = 800
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    
    ###########################################################################
    # CAPA 2: LSTM (Long Short-Term Memory)
    ###########################################################################
    # LSTM(units)
    #
    # ¿Qué hace esta capa?
    # Procesa la secuencia de embeddings de izquierda a derecha,
    # manteniendo un "estado oculto" que actúa como memoria.
    #
    # ¿Cómo funciona internamente?
    # En cada paso temporal t (cada palabra), la LSTM:
    #
    # 1. Recibe:
    #    - xₜ: embedding de la palabra actual (16 dimensiones)
    #    - hₜ₋₁: estado oculto anterior (32 dimensiones)
    #    - cₜ₋₁: cell state (memoria) anterior (32 dimensiones)
    #
    # 2. Calcula 3 puertas (gates) con funciones sigmoid σ:
    #    - iₜ = σ(Wᵢ·xₜ + Uᵢ·hₜ₋₁ + bᵢ)  [input gate: qué agregar]
    #    - fₜ = σ(Wf·xₜ + Uf·hₜ₋₁ + bf)  [forget gate: qué olvidar]
    #    - oₜ = σ(Wₒ·xₜ + Uₒ·hₜ₋₁ + bₒ)  [output gate: qué sacar]
    #
    # 3. Actualiza cell state (memoria a largo plazo):
    #    - c̃ₜ = tanh(Wc·xₜ + Uc·hₜ₋₁ + bc)  [candidato a agregar]
    #    - cₜ = fₜ ⊙ cₜ₋₁ + iₜ ⊙ c̃ₜ          [memoria actualizada]
    #
    # 4. Calcula nuevo estado oculto:
    #    - hₜ = oₜ ⊙ tanh(cₜ)
    #
    # Donde:
    # - W, U: matrices de pesos
    # - b: vectores de bias
    # - ⊙: multiplicación elemento a elemento
    # - σ: sigmoid (rango 0-1)
    # - tanh: tangente hiperbólica (rango -1 a 1)
    #
    # ¿Por qué LSTM y no RNN simple?
    # RNN simple sufre de "vanishing gradient":
    # - No puede recordar información de hace muchos pasos
    # - Los gradientes se hacen muy pequeños durante backpropagation
    #
    # LSTM soluciona esto con:
    # - Cell state: camino directo para información a largo plazo
    # - Gates: control fino sobre qué recordar/olvidar
    #
    # Ejemplo de uso de memoria:
    # Frase: "La comida es buena pero el servicio es malo"
    #         ↓  palabra "pero" activa forget gate
    #         ↓  olvida parcialmente sentimiento positivo previo
    #         ↓  presta atención a lo que viene después
    #
    # Parámetros:
    # - units=32: Tamaño del estado oculto
    # - return_sequences=False (por defecto): Solo devuelve último estado
    #
    # ¿Qué significa return_sequences=False?
    # - True: Devuelve estado oculto para CADA palabra (shape: batch, maxlen, 32)
    #   Uso: Cuando apilamos varias capas LSTM
    # - False: Solo devuelve último estado oculto (shape: batch, 32)
    #   Uso: Cuando vamos a clasificación (nuestro caso)
    #
    # Forma de entrada: (batch, maxlen, embedding_dim) = (batch, 10, 16)
    # Forma de salida: (batch, lstm_units) = (batch, 32)
    #
    # Parámetros totales: 4 × lstm_units × (lstm_units + embedding_dim + 1)
    #                   = 4 × 32 × (32 + 16 + 1)
    #                   = 4 × 32 × 49
    #                   = 6,272 parámetros
    #
    # Estos parámetros corresponden a las 4 operaciones:
    # - Input gate (32 × 49 = 1,568)
    # - Forget gate (32 × 49 = 1,568)
    # - Output gate (32 × 49 = 1,568)
    # - Cell state candidate (32 × 49 = 1,568)
    LSTM(units=lstm_units),
    
    ###########################################################################
    # CAPA 3: Dense (Clasificación final)
    ###########################################################################
    # Dense(1, activation='sigmoid')
    #
    # ¿Qué hace?
    # Transforma el vector de 32 dimensiones (salida de LSTM)
    # en un único valor entre 0 y 1 (probabilidad).
    #
    # Operación matemática:
    # salida = sigmoid(W·h + b)
    #
    # Donde:
    # - h: último estado oculto de LSTM (32 valores)
    # - W: vector de pesos (32 valores)
    # - b: bias (1 valor)
    # - sigmoid: convierte a rango [0, 1]
    #
    # Forma de entrada: (batch, 32)
    # Forma de salida: (batch, 1)
    #
    # Parámetros: 32 + 1 = 33
    #
    # Interpretación:
    # - Salida ≥ 0.5 → Predicción: Positivo (clase 1)
    # - Salida < 0.5 → Predicción: Negativo (clase 0)
    Dense(1, activation='sigmoid')
])

###############################################################################
# COMPILACIÓN DEL MODELO
###############################################################################
# La compilación configura cómo se va a entrenar el modelo

modelo.compile(
    # Loss: Binary Cross Entropy (igual que en MLP)
    # Mide qué tan diferentes son las predicciones de las etiquetas reales
    loss='binary_crossentropy',
    
    # Optimizer: Adam con learning rate por defecto (0.001 en Keras)
    # Nota: En PyTorch usamos lr=0.01, Keras usa 0.001 por defecto
    # Keras tiende a usar valores más conservadores
    optimizer='adam',
    
    # Metrics: Seguimos la accuracy durante el entrenamiento
    # accuracy = (predicciones correctas) / (total de predicciones)
    metrics=['accuracy']
)

print("Arquitectura del modelo:")
modelo.summary()
# La salida de summary() muestra:
# - Nombre de cada capa
# - Forma de salida de cada capa
# - Cantidad de parámetros por capa
#
# Ejemplo de salida esperada:
# Layer (type)                Output Shape              Param #
# =================================================================
# embedding (Embedding)       (None, 10, 16)            800
# lstm (LSTM)                 (None, 32)                6,272
# dense (Dense)               (None, 1)                 33
# =================================================================
# Total params: 7,105

print(f"\nParámetros totales: {modelo.count_params():,}")

# Comparación de complejidad:
# - Perceptrón: 17 parámetros
# - MLP: 145 parámetros
# - LSTM: ~7,105 parámetros (casi 50 veces más que MLP)
#
# ¿Por qué tantos parámetros?
# La LSTM necesita 4 transformaciones por cada paso temporal
# (input gate, forget gate, output gate, cell update)
# Cada una requiere matrices de pesos para combinar
# el input actual (16 dim) con el estado oculto previo (32 dim)

## 6. Entrenamiento

Entrenamos el modelo por varias épocas. La LSTM va a aprender:
- Qué palabras son importantes para el sentimiento
- Cómo el orden afecta el significado
- Qué patrones secuenciales indican positivo o negativo

In [ ]:
###############################################################################
# PARÁMETROS DEL ENTRENAMIENTO
###############################################################################

print("="*60)
print("INICIANDO ENTRENAMIENTO")
print("="*60)

# ¿Por qué 20 épocas?
# Es suficiente para este dataset pequeño (10 frases)
# - Con más datos, usaríamos más épocas (50, 100, etc.)
# - Con early stopping, pararíamos automáticamente cuando no mejore
print(f"Épocas: 20")

# ¿Qué es batch_size y por qué 2?
# El batch size es cuántos ejemplos procesa el modelo antes de actualizar pesos.
#
# Batch size = 2 significa:
# - En cada iteración, procesa 2 frases simultáneamente
# - Calcula gradientes promediando sobre esas 2 frases
# - Actualiza pesos usando ese gradiente promedio
#
# ¿Por qué 2 y no otro número?
# - Muy pequeño (1): "Stochastic Gradient Descent", ruidoso pero actualiza rápido
# - Medio (2-32): Buen balance, gradientes más estables
# - Grande (128, 256): Más eficiente computacionalmente, pero necesita más datos
#
# Con 10 ejemplos y batch_size=2:
# - Cada época procesa 10/2 = 5 batches
# - 20 épocas × 5 batches = 100 actualizaciones de pesos totales
#
# Comparación con laboratorios anteriores:
# - Perceptrón: Actualizaba después de CADA ejemplo (batch_size=1)
# - MLP: Procesaba TODOS los ejemplos juntos (batch_size=10)
# - LSTM: Procesa en mini-batches (batch_size=2)
#
# ¿Por qué mini-batches son mejores?
# - Más eficientes que batch_size=1 (menos actualizaciones ruidosas)
# - Mejor generalización que batch_size=total (no memoriza tanto)
# - Aprovecha mejor GPUs (operaciones vectorizadas)
print(f"Batch size: 2\n")

###############################################################################
# ENTRENAMIENTO CON model.fit()
###############################################################################
# En Keras, model.fit() hace TODO el ciclo de entrenamiento automáticamente:
# 1. Divide datos en batches
# 2. Para cada época:
#    - Para cada batch:
#      a. Forward pass
#      b. Calcula loss
#      c. Backward pass (backpropagation)
#      d. Actualiza pesos
#    - Reporta métricas (loss, accuracy)
#
# Es equivalente al bucle manual que hicimos en PyTorch,
# pero más compacto y optimizado.
#
###############################################################################

history = modelo.fit(
    # X: Datos de entrada (10 frases × 10 palabras)
    # y: Etiquetas (10 valores, 0 o 1)
    X, y,
    
    # epochs: Cantidad de veces que recorre todo el dataset
    epochs=20,
    
    # batch_size: Cantidad de ejemplos por batch
    # Con 10 ejemplos y batch_size=2, habrá 5 batches por época
    batch_size=2,
    
    # verbose: Nivel de información durante entrenamiento
    # 0 = silencioso
    # 1 = barra de progreso (elegido)
    # 2 = una línea por época
    verbose=1
    
    # Parámetros adicionales que podríamos usar:
    # validation_split=0.2  → Reserva 20% para validación
    # validation_data=(X_val, y_val)  → Usa datos de validación externos
    # callbacks=[EarlyStopping(...)]  → Para cuando no mejore
)

# El objeto 'history' contiene el historial de entrenamiento:
# - history.history['loss']: pérdida en cada época
# - history.history['accuracy']: accuracy en cada época
# - history.history['val_loss']: pérdida de validación (si usamos validation)
# - history.history['val_accuracy']: accuracy de validación

print("\n" + "="*60)
print("ENTRENAMIENTO FINALIZADO")
print("="*60)

# ¿Qué pasó durante el entrenamiento?
#
# Época 1:
# - Embeddings aleatorios, LSTM sin entrenar
# - Loss alta (~0.7), accuracy baja (~50%, como adivinar al azar)
#
# Épocas intermedias (5-10):
# - Embeddings empiezan a capturar polaridad (positivo/negativo)
# - LSTM aprende patrones secuenciales
# - Loss baja (~0.3-0.4), accuracy mejora (~70-80%)
#
# Épocas finales (15-20):
# - Modelo converge
# - Loss muy baja (~0.1 o menos), accuracy alta (~90-100%)
# - Puede haber overfitting si accuracy = 100% (memoriza training data)
#
# Señales de buen entrenamiento:
# ✓ Loss disminuye consistentemente
# ✓ Accuracy aumenta consistentemente
# ✓ No hay saltos erráticos en las métricas
#
# Señales de problemas:
# ✗ Loss aumenta o se estanca
# ✗ Accuracy no mejora
# ✗ Valores NaN en loss (gradientes explotaron)

## 7. Análisis del entrenamiento

Observá cómo evolucionan la pérdida (loss) y la precisión (accuracy) durante el entrenamiento.

- **Loss decrece**: El modelo comete menos errores
- **Accuracy aumenta**: Más predicciones correctas

Si la accuracy llega a 1.0, significa que el modelo clasificó perfectamente todos los ejemplos de entrenamiento.

## 8. Evaluación con frases nuevas

Ahora vamos a probar el modelo con frases que no vio durante el entrenamiento. Esta es la verdadera prueba de si aprendió patrones generalizables.

In [ ]:
frases_nuevas = [
    "Muy buena atención, quedé encantado",
    "Horrible experiencia, no vuelvo más",
    "Todo excelente, gracias por la atención",
    "Me arrepiento completamente, fue un desastre",
    "Un servicio impecable y rápido"
]

print("="*60)
print("EVALUACIÓN EN FRASES NUEVAS")
print("="*60)

# Tokenizamos y aplicamos padding
secuencias_nuevas = tokenizer.texts_to_sequences(frases_nuevas)
X_nuevo = pad_sequences(secuencias_nuevas, maxlen=maxlen, padding='post')

# Predicción
predicciones = modelo.predict(X_nuevo, verbose=0)

# Mostrar resultados
for i, (frase, pred) in enumerate(zip(frases_nuevas, predicciones), 1):
    probabilidad = pred[0]
    clase = "Positivo" if probabilidad >= 0.5 else "Negativo"
    
    print(f"\nFrase {i}: '{frase}'")
    print(f"  Predicción: {clase} (probabilidad: {probabilidad:.2f})")
    
    # Indicador de confianza
    if probabilidad >= 0.8 or probabilidad <= 0.2:
        print(f"  Confianza: Alta")
    elif probabilidad >= 0.6 or probabilidad <= 0.4:
        print(f"  Confianza: Media")
    else:
        print(f"  Confianza: Baja (ambiguo)")

print("\n" + "="*60)

## 9. Comparación con enfoques anteriores

Recapitulemos lo que mejoramos con cada modelo:

### Perceptrón simple:
- ✗ No considera orden de palabras
- ✗ Representación binaria (0/1)
- ✗ Modelo lineal
- ✓ Muy simple de entender

### MLP (Red Multicapa):
- ✗ No considera orden de palabras
- ✗ Representación binaria (0/1)
- ✓ Puede aprender patrones no lineales
- ✓ Mejor capacidad de generalización

### LSTM:
- ✓ **Considera el orden de las palabras**
- ✓ **Embeddings aprendidos** (vectores densos)
- ✓ **Memoria del contexto** (puede recordar palabras anteriores)
- ✓ Puede aprender patrones no lineales complejos

La LSTM es un avance significativo porque finalmente podemos procesar el lenguaje como una secuencia, no como una bolsa desordenada de palabras.

## 10. Reflexión final

### ¿Qué aprendimos?

1. **Procesamiento de secuencias**: Las LSTM pueden leer frases palabra por palabra, manteniendo memoria del contexto.

2. **Embeddings de palabras**: En lugar de vectores binarios (0/1), cada palabra se representa con un vector denso que captura su significado.

3. **Orden importa**: "No me gusta" y "Me gusta, no" ahora se procesan diferente (antes eran idénticos con bag-of-words).

4. **Tokenización automática**: Keras construye el vocabulario automáticamente y maneja palabras desconocidas con `<OOV>`.

### Ventajas sobre MLP con bag-of-words:

- Captura el orden de las palabras
- Aprende representaciones semánticas (embeddings)
- Puede detectar patrones secuenciales
- Mejor manejo de frases largas

### Limitaciones que aún persisten:

1. **Procesamiento secuencial**: La LSTM lee de izquierda a derecha, puede "olvidar" información del principio en frases muy largas

2. **No puede mirar hacia adelante**: Al procesar una palabra, no sabe qué viene después

3. **Dataset pequeño**: Con solo 10 ejemplos, los embeddings no se entrenan bien

4. **Vocabulario limitado**: Solo conoce las palabras que aparecieron en el entrenamiento

### ¿Qué sigue?

En la próxima actividad vamos a ver cómo los **modelos preentrenados** como BETO (BERT en español) resuelven muchas de estas limitaciones:

- Ya fueron entrenados con millones de textos
- Tienen embeddings muy ricos
- Usan arquitectura Transformer (no secuencial, con **atención**)
- Pueden hacer análisis de sentimiento sin necesidad de entrenar desde cero

Esto nos va a llevar al concepto de **transfer learning**, que revolucionó el NLP y es la base de los LLMs modernos como GPT.